In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
from zipfile import ZipFile
train_path = '/content/drive/My Drive/amazon.zip' 
with ZipFile (train_path , "r") as zip:
  zip.extractall()
  print("Finsih")

Finsih


In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [ ]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="/content/drive/My Drive/train",target_size=(150,150))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/content/amazon/testing", target_size=(150,150))

Found 1700 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


In [ ]:
nb_train_samples=1700
nb_validation_samples=30
batch_size=50
epochs=20

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(150,150,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=3, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 75, 75, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 37, 37, 256)      

# Using SGD+Momentum

In [ ]:
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD

checkpoint= ModelCheckpoint("Model.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
momentums = [0.0, 0.5, 0.9, 0.99]
opt = SGD(lr=0.01, momentum=0.5, decay=0.01)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
34/34 [==============================] - 471s 14s/step - loss: 1.0824 - accuracy: 0.4519 - val_loss: 1.0940 - val_accuracy: 0.3333

Epoch 00001: val_accuracy improved from -inf to 0.33333, saving model to Model.h5
Epoch 2/20
34/34 [==============================] - 277s 8s/step - loss: 1.0474 - accuracy: 0.4835 - val_loss: 1.2927 - val_accuracy: 0.3333

Epoch 00002: val_accuracy did not improve from 0.33333
Epoch 3/20
34/34 [==============================] - 15s 447ms/step - loss: 0.9296 - accuracy: 0.5538 - val_loss: 1.0431 - val_accuracy: 0.6000

Epoch 00003: val_accuracy improved from 0.33333 to 0.60000, saving model to Model.h5
Epoch 4/20
34/34 [==============================] - 16s 459ms/step - loss: 0.6868 - accuracy: 0.6746 - val_loss: 0.5533 - val_accuracy: 0.6667

Epoch 00004: val_accuracy improved from 0.60000 to 0.66667, saving model to Model.h5
Epoch 5/20
34/34 [==============================] - 15s 451ms/step - loss: 0.5282 - accuracy: 0.7377 - val_loss: 0.5869 

# Using Adam optimiser

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3),input_shape=(150, 150, 3), activation = 'relu'))
model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size=(3,3),activation = 'relu'))
model.add(Conv2D(64, kernel_size=(3,3),activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(units=3, activation = "softmax"))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 73, 73, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 69, 69, 64)       

In [ ]:
checkpoint= ModelCheckpoint("Model.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
momentums = [0.0, 0.5, 0.9, 0.99]
opt = Adam(lr=0.001)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
34/34 [==============================] - 9s 258ms/step - loss: 3.6216 - accuracy: 0.7206 - val_loss: 142.9827 - val_accuracy: 0.3333

Epoch 00001: val_accuracy improved from -inf to 0.33333, saving model to Model.h5
Epoch 2/20
34/34 [==============================] - 6s 180ms/step - loss: 3.4551 - accuracy: 0.7736 - val_loss: 61.5959 - val_accuracy: 0.3333

Epoch 00002: val_accuracy did not improve from 0.33333
Epoch 3/20
34/34 [==============================] - 6s 167ms/step - loss: 2.7899 - accuracy: 0.8000 - val_loss: 12.3108 - val_accuracy: 0.7000

Epoch 00003: val_accuracy improved from 0.33333 to 0.70000, saving model to Model.h5
Epoch 4/20
34/34 [==============================] - 6s 170ms/step - loss: 2.2222 - accuracy: 0.8024 - val_loss: 4.9279 - val_accuracy: 0.5667

Epoch 00004: val_accuracy did not improve from 0.70000
Epoch 5/20
34/34 [==============================] - 6s 168ms/step - loss: 1.3073 - accuracy: 0.8387 - val_loss: 2.5695 - val_accuracy: 0.7667

Epoc

# Using RMSprop optimizer

In [ ]:
from keras.optimizers import RMSprop

In [ ]:
checkpoint= ModelCheckpoint("Model.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
opt = RMSprop(lr=0.001)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
34/34 [==============================] - 7s 208ms/step - loss: 2.0157 - accuracy: 0.8396 - val_loss: 94.0608 - val_accuracy: 0.3667

Epoch 00001: val_accuracy improved from -inf to 0.36667, saving model to Model.h5
Epoch 2/20
34/34 [==============================] - 6s 166ms/step - loss: 1.1586 - accuracy: 0.8483 - val_loss: 5.9573 - val_accuracy: 0.6000

Epoch 00002: val_accuracy improved from 0.36667 to 0.60000, saving model to Model.h5
Epoch 3/20
34/34 [==============================] - 5s 160ms/step - loss: 1.0693 - accuracy: 0.8472 - val_loss: 2.2761 - val_accuracy: 0.7000

Epoch 00003: val_accuracy improved from 0.60000 to 0.70000, saving model to Model.h5
Epoch 4/20
34/34 [==============================] - 6s 166ms/step - loss: 0.8589 - accuracy: 0.8778 - val_loss: 1.4053 - val_accuracy: 0.6667

Epoch 00004: val_accuracy did not improve from 0.70000
Epoch 5/20
34/34 [==============================] - 6s 163ms/step - loss: 0.7215 - accuracy: 0.8764 - val_loss: 1.7360 -

# Using Nadam Optimizer

In [ ]:
from keras.optimizers import Nadam

In [ ]:
checkpoint= ModelCheckpoint("Model.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
opt = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
34/34 [==============================] - 8s 241ms/step - loss: 0.6762 - accuracy: 0.8851 - val_loss: 2.9882 - val_accuracy: 0.7000

Epoch 00001: val_accuracy improved from -inf to 0.70000, saving model to Model.h5
Epoch 2/20
34/34 [==============================] - 6s 187ms/step - loss: 0.8930 - accuracy: 0.8972 - val_loss: 0.7477 - val_accuracy: 0.8000

Epoch 00002: val_accuracy improved from 0.70000 to 0.80000, saving model to Model.h5
Epoch 3/20
34/34 [==============================] - 6s 185ms/step - loss: 0.7425 - accuracy: 0.8906 - val_loss: 32.7597 - val_accuracy: 0.5000

Epoch 00003: val_accuracy did not improve from 0.80000
Epoch 4/20
34/34 [==============================] - 7s 192ms/step - loss: 0.6705 - accuracy: 0.8971 - val_loss: 0.5008 - val_accuracy: 0.8333

Epoch 00004: val_accuracy improved from 0.80000 to 0.83333, saving model to Model.h5
Epoch 5/20
34/34 [==============================] - 6s 186ms/step - loss: 1.3483 - accuracy: 0.8830 - val_loss: 1696.091

# Nesterov Accelerated Gradient Descent

In [ ]:
from keras.optimizers import SGD

checkpoint= ModelCheckpoint("Model.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
momentums = [0.0, 0.5, 0.9, 0.99]
opt = SGD(lr=0.001, momentum=0.0, decay=0.01,nesterov=True)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
34/34 [==============================] - 7s 197ms/step - loss: 0.2848 - accuracy: 0.9123 - val_loss: 8.6605 - val_accuracy: 0.9000

Epoch 00001: val_accuracy improved from -inf to 0.90000, saving model to Model.h5
Epoch 2/20
34/34 [==============================] - 5s 152ms/step - loss: 0.4131 - accuracy: 0.8915 - val_loss: 0.2333 - val_accuracy: 0.9667

Epoch 00002: val_accuracy improved from 0.90000 to 0.96667, saving model to Model.h5
Epoch 3/20
34/34 [==============================] - 5s 149ms/step - loss: 0.2628 - accuracy: 0.9142 - val_loss: 66.9409 - val_accuracy: 0.5000

Epoch 00003: val_accuracy did not improve from 0.96667
Epoch 4/20
34/34 [==============================] - 5s 151ms/step - loss: 0.3475 - accuracy: 0.9160 - val_loss: 12.4678 - val_accuracy: 0.8000

Epoch 00004: val_accuracy did not improve from 0.96667
Epoch 5/20
34/34 [==============================] - 5s 152ms/step - loss: 0.2831 - accuracy: 0.9210 - val_loss: 5.4653 - val_accuracy: 0.8667

Epoch 